<h1 color="green">EDA Grammys</h1>

In [21]:
import sys
sys.path.append("../DB_Connection")
import Pysqlconnect
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [4]:
def workers_separate(wstring:str)->list:
    """This function is for separate the job and the persons in the """
    if type(wstring) != str:
        return wstring
    try:
        wslist = wstring.split(sep=";") #transform the string to list -> "name, role"
        wslist = [i if i[0]!=" " else i[1:] for i in wslist]
        wsdict = {i.split(sep=", ")[1]:i.split(sep=", ")[0] for i in wslist} #[0] is the name of the role [1]
        for i in wsdict.keys(): #iterate the dictionary values for crate a list if the persons on one job are > 1
            if "&" in wsdict[i]:
                wsdict[i] = wsdict[i].split(sep=" & ")
                continue
            wsdict[i] = [wsdict[i]]    
    except Exception as error:
        print(error)
        print(wstring) #look for strange thinks
        return
    return wsdict

In [24]:
def datetime_transform(stdate:str)->datetime:
    date = datetime.strptime(stdate, '%Y-%m-%dT%H:%M:%S%z')
    formatdate = date.strftime('%Y-%m-%d')
    return formatdate


In [6]:
df_grammys = pd.read_sql("SELECT * FROM grammys;", Pysqlconnect.connection())
#Load the data from SQL database

In [7]:
df_grammys.head(5)

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,True
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,True
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,True
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,True
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,True


In [8]:
dfshape = df_grammys.shape
print(f"Dataset rows : {dfshape[0]}", f"Dataset columns : {dfshape[1]}", sep="\n")

Dataset rows : 4810
Dataset columns : 10


In [9]:
df_grammys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4810 entries, 0 to 4809
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          4810 non-null   int64 
 1   title         4810 non-null   object
 2   published_at  4810 non-null   object
 3   updated_at    4810 non-null   object
 4   category      4810 non-null   object
 5   nominee       4804 non-null   object
 6   artist        2970 non-null   object
 7   workers       2620 non-null   object
 8   img           3443 non-null   object
 9   winner        4810 non-null   bool  
dtypes: bool(1), int64(1), object(8)
memory usage: 343.0+ KB


We observe the dtypes of our dataframe and some info about the columns nulls state,
the columns types looks good except (published_at, updated_at) these must are datetime dtypes
but i will decide if is a good idea change these types and the workers columns can have a transformation for analysis
and we can see that the artist have a lot of nulls

In [10]:
df_grammys.index

RangeIndex(start=0, stop=4810, step=1)

We look the consistency of the index although it shouldn't be bad.

---

In [11]:
print(f"Data entries : {df_grammys.shape[0]}  | Data entries without duplicates : {df_grammys[['year','artist','category','nominee']].duplicated().value_counts()[0]}")

Data entries : 4810  | Data entries without duplicates : 4810


We can see that we do not have duplicates if we look at the categories that will be repeated if a disk were duplicated
year of the grammys, artist of the song, category and finally the nominee 

In [12]:
for i in df_grammys.keys():
    print(f"{i} nulls : {4810 - df_grammys[i].notnull().value_counts()[0]}")

year nulls : 0
title nulls : 0
published_at nulls : 0
updated_at nulls : 0
category nulls : 0
nominee nulls : 6
artist nulls : 1840
workers nulls : 2190
img nulls : 1367
winner nulls : 0


We see that workers have a lot of nulls entries and artist colum have a lot of nulls entries

---

In [13]:
df_grammys["category"].nunique()
#look for how many categories are

638

In [14]:
df_grammys.describe(include="object")

,title,published_at,updated_at,category,nominee,artist,workers,img
count,4810,4810,4810,4810,4804,2970,2620,3443
unique,62,4,10,638,4131,1658,2366,1463
top,62nd Annual GRAMMY Awards (2019),2017-11-28T00:03:45-08:00,2019-09-10T01:08:19-07:00,Song Of The Year,Bridge Over Troubled Water,(Various Artists),"John Williams, composer (John Williams)",https://www.grammy.com/sites/com/files/styles/...
freq,433,4205,778,70,7,66,20,26


Describe the categorical columns and we can observe on published_at most of the data was publish at 2017 

In [15]:
df_grammys["workers_test"] = df_grammys["workers"].apply(workers_separate)
#create the column workers_test to analyze the data compliance
#the column meets the respective job and the names

list index out of range
Kirk Franklin; Kirk Franklin, songwriter
list index out of range
Gloria Gaynor Featuring Yolanda Adams; Bryan Fowler, Gloria Gaynor & Chris Stevens, songwriters
list index out of range
Travis Greene Featuring Jekalyn Carr
list index out of range
Koryn Hawthorne Featuring Natalie Grant
list index out of range
Tasha Cobbs Leonard; Tony Brown, Brandon Lake, Tasha Cobbs Leonard & Nate Moore, songwriters
list index out of range
for KING & COUNTRY & Dolly Parton; Josh Kerr, Jordan Reynolds, Joel Smallbone, Luke Smallbone & Tedd Tjornhom, songwriters
list index out of range
Casting Crowns; Mark Hall, Bernie Herms & Matthew West, songwriters
list index out of range
Danny Gokey; Danny Gokey, Ethan Hulse & Colby Wedgeworth, songwriters
list index out of range
Tauren Wells
list index out of range
Zach Williams; Ethan Hulse, Andrew Ripp, Jonathan Smith & Zach Williams, songwriters
list index out of range
• Arizona Baby (Kevin Abstract) (A)  • Lover (Taylor Swift) (A)  • Nor

In [16]:
df_grammys.head(5)

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner,workers_test
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,True,"{'producer': ['Finneas O'Connell'], 'engineers..."
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,True,"{'Brad Cook': ['BJ Burton'], 'Zach Hanson & Ch..."
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,True,"{'Tommy Brown': ['Charles Anderson'], 'John Ha..."
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,True,"{'producer': ['Rodney “Darkchild” Jerkins'], '..."
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,True,"{'producers': ['Disclosure', 'Denis Kosiak'], ..."


After try to do the workers column transformation i discover that the data in this column dont have the consistency necesari to have a good analisys for job 

In [17]:
df_grammys.drop(columns=["workers_test"], inplace=True) #Drop the column workers_test
df_grammys.head(5)

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,True
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,True
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,True
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,True
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,True


---

In [26]:
df_grammys["published_date"] = df_grammys["published_at"].apply(datetime_transform)

In [27]:
df_grammys.head(5)

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner,published_date
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,True,2020-05-19
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,True,2020-05-19
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,True,2020-05-19
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,True,2020-05-19
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,True,2020-05-19


In [28]:
df_grammys.describe(include="object")

,title,published_at,updated_at,category,nominee,artist,workers,img,published_date
count,4810,4810,4810,4810,4804,2970,2620,3443,4810
unique,62,4,10,638,4131,1658,2366,1463,4
top,62nd Annual GRAMMY Awards (2019),2017-11-28T00:03:45-08:00,2019-09-10T01:08:19-07:00,Song Of The Year,Bridge Over Troubled Water,(Various Artists),"John Williams, composer (John Williams)",https://www.grammy.com/sites/com/files/styles/...,2017-11-28
freq,433,4205,778,70,7,66,20,26,4205


In [29]:
df_grammys.published_date.unique()

array(['2020-05-19', '2018-12-06', '2017-11-28', '2018-05-22'],
      dtype=object)

We observe some strange think althoug we have 4810 entries whit a lot of differents songs most of the published dates are the same day (2017-11-28)

In [35]:
#look for the songs in this date 2017-11-28
df_grammys[df_grammys.published_date == "2017-11-28"]["winner"].value_counts()

winner
True    4205
Name: count, dtype: int64

all of the candidates in this date won the grammys that's weird

In [36]:
df_grammys["winner"].value_counts()

winner
True    4810
Name: count, dtype: int64

how we can see this dataset its about grammy winners that is interesant we can rm the column if we dont have to do anything in the merge

# Before drop columns i'm going to see the Spotify data